In [ ]:
from Plan import CreateSamples
from Prepare import OT2Commands as ALH
from opentrons import simulate, execute, protocol_api
from Prepare import OT2_Hcell_commands as HC

import pandas as pd
import time 
import numpy as np

import string

In [ ]:
#if needed
#import importlib
#importlib.reload(HC)

In [ ]:
labware_dir_path = r"./Custom Labware/"
custom_labware_dict = ALH.custom_labware_dict(labware_dir_path)

In [ ]:
path = "./H_cell_protocol.csv"
chem_path = r"Chemical Database.csv"
plan = CreateSamples.get_experiment_plan(path, chem_path)

## Experiment

In [ ]:
protocol = execute.get_protocol_api('2.8', extra_labware=custom_labware_dict)
loaded_dict = ALH.loading_labware(protocol, plan, well_order='row')

In [ ]:
#if using 8 h-cells, add sample names for H5, H6, H7, H8

log_metadata={
    'author': 'Author Name',
    'sample_name':'Analyte Name',
    'hcell_membrane':'H1: sample_1_name, H2: sample_2_name, H3: sample_3_name, H4: sample_4_name',
    'solvent':'water',
    'restock_solvent':'water'
}

In [ ]:
aliquot_volume= 20 # uL
sample_dilution_volume = 180 #uL
time_schedule, delay_time = HC.get_time_schedule(66, [66], [6]) # in hrs

h_cell, wellplates= HC.get_Hcell_labware(loaded_dict)
h_cell_name , h_cell_dict = HC.get_hcell_name(h_cell)

sample_schedule = HC.get_sample_schedule(h_cell_name, wellplates, time_schedule, delay_time)

In [ ]:
HC.H_cell_protocol(protocol, loaded_dict,
                    sample_schedule, h_cell_dict, aliquot_volume=aliquot_volume,
                    sample_dilution_volume=sample_dilution_volume,
                    max_stock_vol= 17000, log_file_info= log_metadata, log_filename= 'name_of_experiment')